In [1]:
import matlab.engine
import os
import numpy as np
from tqdm import trange
import pickle

In [2]:
test_cases = [
    'pglib_opf_case24_ieee_rts.m', 
    'pglib_opf_case73_ieee_rts.m',
    'pglib_opf_case162_ieee_dtc.m',
    'pglib_opf_case300_ieee.m',
]

In [3]:
def get_Pb_lim(Pg_lim, GB_map):
    Pb_lim = {}

    for g_idx in range(len(GB_map)):
        b_idx = GB_map[g_idx]
        try:
            Pb_lim[b_idx][0] = Pb_lim[b_idx][0] + Pg_lim[g_idx][0]
            Pb_lim[b_idx][1] = Pb_lim[b_idx][1] + Pg_lim[g_idx][1]
        except KeyError:
            Pb_lim[b_idx] = np.zeros(2)
            Pb_lim[b_idx][0] = Pg_lim[g_idx][0]
            Pb_lim[b_idx][1] = Pg_lim[g_idx][1]
    return Pb_lim


def get_Pb(Pg, GB_map):
    Pb = {}

    for g_idx in range(len(GB_map)):
        b_idx = GB_map[g_idx]
        try:
            Pb[b_idx] = Pb[b_idx] + Pg[g_idx]
        except KeyError:
            Pb[b_idx] = Pg[g_idx]
    return Pb

In [4]:
def get_Pg_active_constraints(Pg, Pg_lim):
    active_constraints = np.zeros((len(Pg), 2))

    for g_idx in range(Pg_lim.shape[0]):
        if Pg[g_idx] >= Pg_lim[g_idx][0]:
            active_constraints[g_idx][0] = 1
        if Pg[g_idx] <= Pg_lim[g_idx][1]:
            active_constraints[g_idx][1] = 1

    return np.sum(active_constraints, axis=1) % 2


def get_Pb_active_constraints(Pg, Pg_lim, GB_map):
    Pb = get_Pb(Pg, GB_map)
    Pb_lim = get_Pb_lim(Pg_lim, GB_map)
    
    Bg_idx = list(Pb.keys())
    
    active_constraints = np.zeros((len(Bg_idx), 2))
    
    for i in range(len(Bg_idx)):
        b_idx = Bg_idx[i]
        if Pb[b_idx] >= Pb_lim[b_idx][0]:
            active_constraints[i][0] = 1
        if Pb[b_idx] <= Pb_lim[b_idx][1]:
            active_constraints[i][1] = 1
    
    return np.sum(active_constraints, axis=1) % 2


def get_F_active_constraints(F, F_lim):
    active_constraints = []

    for f, f_lim in zip(F, F_lim):
        if abs(f) >= abs(f_lim):
            active_constraints.append(1)
        else:
            active_constraints.append(0)

    return np.array(active_constraints)


def merge_active_constraints(Pg_active, Pb_active, F_active):
    return np.array(list(Pg_active) + list(Pb_active) + list(F_active))

In [41]:
def create_dataset(file_name, dataset_size, std_scaler=0.03):
    print("> creating dataset with {}".format(file_name))

    x = []
    y = []

    org_dir = os.getcwd()
    os.chdir('./matpower7.0/')

    eng = matlab.engine.start_matlab()

    data = eng.dc_opf_solver(file_name, 0.03)

    Pg = np.squeeze(np.array(data['Pg']))
    B_idx = np.squeeze(np.array(data['B_idx']))
    GB_map = np.squeeze(np.array(data['GB_map'])).astype(int)
    F = np.squeeze(np.array(data['F']))
    Pg_lim = np.array(data['Pg_lim'])
    F_lim = np.array(data['F_lim'])
    w = np.squeeze(np.array(data['w']))

    for i in trange(dataset_size):
        data = eng.dc_opf_solver(file_name, std_scaler)
        # assing x data
        x.append(np.array(data['w']))
        # assgin y data
        Pg_active = get_Pg_active_constraints(Pg, Pg_lim)
        Pb_active = get_Pb_active_constraints(Pg, Pg_lim, GB_map)
        F_active = get_F_active_constraints(F, F_lim)
        active_constraints = merge_active_constraints(Pg_active, Pb_active,
                                                      F_active)
        y.append(np.array(active_constraints))

    eng.quit()

    os.chdir(org_dir)

    return {'x': np.squeeze(np.array(x)), 'y': np.array(y)}

## Example of parsing active constraints

In [21]:
org_dir = os.getcwd()
os.chdir('./matpower7.0/')

In [23]:
eng = matlab.engine.start_matlab()

In [24]:
test_case_idx = 0
case_name = test_cases[test_case_idx]

In [25]:
data = eng.dc_opf_solver(case_name, 0.03)

In [26]:
Pg = np.squeeze(np.array(data['Pg']))
B_idx = np.squeeze(np.array(data['B_idx']))
GB_map = np.squeeze(np.array(data['GB_map'])).astype(int)
F = np.squeeze(np.array(data['F']))
Pg_lim = np.array(data['Pg_lim'])
F_lim = np.array(data['F_lim'])
w = np.squeeze(np.array(data['w']))

### Generator active constraints: [Pg_min, Pg_max]]

In [27]:
Pg

array([ 16.        ,  16.        ,  76.        ,  76.        ,
        16.        ,  16.        ,  76.        ,  76.        ,
        42.06152311,  42.06152311,  42.06152311,  69.        ,
        69.        ,  69.        ,   0.        ,   2.4       ,
         2.4       ,   2.4       ,   2.4       ,   2.4       ,
       155.        , 155.        , 400.        , 400.        ,
        50.        ,  50.        ,  50.        ,  50.        ,
        50.        ,  50.        , 155.        , 155.        ,
       350.        ])

In [28]:
Pg_lim

array([[ 20. ,  16. ],
       [ 20. ,  16. ],
       [ 76. ,  15.2],
       [ 76. ,  15.2],
       [ 20. ,  16. ],
       [ 20. ,  16. ],
       [ 76. ,  15.2],
       [ 76. ,  15.2],
       [100. ,  25. ],
       [100. ,  25. ],
       [100. ,  25. ],
       [197. ,  69. ],
       [197. ,  69. ],
       [197. ,  69. ],
       [  0. ,   0. ],
       [ 12. ,   2.4],
       [ 12. ,   2.4],
       [ 12. ,   2.4],
       [ 12. ,   2.4],
       [ 12. ,   2.4],
       [155. ,  54.3],
       [155. ,  54.3],
       [400. , 100. ],
       [400. , 100. ],
       [ 50. ,  10. ],
       [ 50. ,  10. ],
       [ 50. ,  10. ],
       [ 50. ,  10. ],
       [ 50. ,  10. ],
       [ 50. ,  10. ],
       [155. ,  54.3],
       [155. ,  54.3],
       [350. , 140. ]])

In [29]:
Pg_active = get_Pg_active_constraints(Pg, Pg_lim)
Pg_active

array([1., 1., 1., 1., 1., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])

### Bus active constraints, due to precense of co-related generation: [Pb_min, Pb_max]

In [30]:
Pb = get_Pb(Pg, GB_map)
Pb

{1: 184.0,
 2: 184.0,
 7: 126.1845693377164,
 13: 207.00000000001955,
 14: 0.0,
 15: 167.0,
 16: 155.0,
 18: 400.0,
 21: 400.0,
 22: 300.0,
 23: 660.0}

In [31]:
Pb_lim = get_Pb_lim(Pg_lim, GB_map)
Pb_lim

{1: array([192. ,  62.4]),
 2: array([192. ,  62.4]),
 7: array([300.,  75.]),
 13: array([591., 207.]),
 14: array([0., 0.]),
 15: array([215. ,  66.3]),
 16: array([155. ,  54.3]),
 18: array([400., 100.]),
 21: array([400., 100.]),
 22: array([300.,  60.]),
 23: array([660. , 248.6])}

In [32]:
Pb_active = get_Pb_active_constraints(Pg, Pg_lim, GB_map)
Pb_active

array([0., 0., 0., 0., 0., 0., 1., 1., 1., 1., 1.])

### Flow Constraints: [-F_max, F_max] 

In [33]:
F

array([   9.01807878,   -2.8249142 ,   71.54887695,   43.61314964,
         57.74153453,   27.6901447 , -222.71283194,  -28.47282559,
          1.22784691,  -83.59383372,   -0.44140075,  -95.30762768,
        -74.37610955, -119.27959575, -133.02412889, -160.83080313,
       -174.7100866 ,  -97.62174494, -182.48865394,  -72.66877603,
       -235.06543946, -222.52355738, -376.8556333 ,   86.97128895,
       -226.44345794, -226.44345794,  222.71283194, -328.99516323,
         93.49592069, -187.92434861, -141.07081462,  -53.02113475,
        -53.02113475,  -39.92651101,  -39.92651101, -101.20550158,
       -101.20550158, -158.92918538])

In [34]:
F_lim

array([[175.],
       [175.],
       [175.],
       [175.],
       [175.],
       [175.],
       [400.],
       [175.],
       [175.],
       [175.],
       [175.],
       [175.],
       [175.],
       [400.],
       [400.],
       [400.],
       [400.],
       [500.],
       [500.],
       [500.],
       [500.],
       [500.],
       [500.],
       [500.],
       [500.],
       [500.],
       [500.],
       [500.],
       [500.],
       [500.],
       [500.],
       [500.],
       [500.],
       [500.],
       [500.],
       [500.],
       [500.],
       [500.]])

In [35]:
F_active = get_F_active_constraints(F, F_lim)
F_active

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

### Merge all active constriants

In [36]:
active_aconstraints = merge_active_constraints(Pg_active, Pb_active, F_active)
active_aconstraints

array([1., 1., 1., 1., 1., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0.,
       0., 0., 0., 0., 0., 1., 1., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [37]:
eng.quit()

In [43]:
os.chdir(org_dir)

## Create Datasets for NNs

### example of creating a dataset
- **x**: uncertainty realization as a feature input
- **y**: active constraints index as a label output

In [47]:
dataset = create_dataset(case_name, dataset_size=1)
dataset

> creating dataset with pglib_opf_case24_ieee_rts.m


100%|██████████| 1/1 [00:00<00:00,  3.30it/s]


{'x': array([ 1.58401582,  3.01095666,  3.92517972, -0.67363885,  0.62594622,
        -3.21211384,  3.33148362, -5.88446965, -5.61156991, -4.73556736,
         0.        ,  0.        ,  2.58526479, -4.39368282, 13.03153912,
        -5.13454926,  0.        , -2.41205595,  1.73329259,  1.20137701,
         0.        ,  0.        ,  0.        ,  0.        ]),
 'y': array([[1., 1., 1., 1., 1., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 0., 0., 0., 0., 0., 0., 1., 1., 1., 1., 1., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0.]])}

### Build datasets

In [59]:
def save_dataset(test_case, dataset):
    file_name = test_case.split('.')[0]
    file_path = './datasets/'
    file_dir = file_path + file_name + '.pickle'
    outfile = open(file_dir,'wb')
    pickle.dump(dataset, outfile)
    outfile.close()

    
def build_datasets(test_cases, dataset_size):
    for test_case in test_cases:
        # create a dataset
        dataset = create_dataset(test_case, dataset_size)
        # save the dataset
        save_dataset(test_case, dataset)

In [60]:
# dataset_size = 2
dataset_size = 50000

In [ ]:
build_datasets(test_cases, dataset_size)

> creating dataset with pglib_opf_case24_ieee_rts.m


100%|██████████| 50000/50000 [1:24:19<00:00,  9.88it/s]


> creating dataset with pglib_opf_case73_ieee_rts.m


100%|██████████| 50000/50000 [3:19:10<00:00,  4.18it/s]  


> creating dataset with pglib_opf_case162_ieee_dtc.m


 31%|███       | 15425/50000 [1:04:37<2:24:05,  4.00it/s]

### Check datasets

In [57]:
infile = open('./datasets/pglib_opf_case24_ieee_rts.pickle','rb')
dataset = pickle.load(infile)
infile.close()

In [58]:
dataset

{'x': array([[ 1.58401582,  3.01095666,  3.92517972, -0.67363885,  0.62594622,
         -3.21211384,  3.33148362, -5.88446965, -5.61156991, -4.73556736,
          0.        ,  0.        ,  2.58526479, -4.39368282, 13.03153912,
         -5.13454926,  0.        , -2.41205595,  1.73329259,  1.20137701,
          0.        ,  0.        ,  0.        ,  0.        ],
        [ 3.54218077,  3.2279853 , -4.66372524,  0.17173718, -2.5860693 ,
         -4.54308303, -0.02568498,  7.86239348, -4.04074605,  2.17256605,
          0.        ,  0.        , -8.65806115,  0.18948444,  5.25453197,
          3.30183065,  0.        ,  0.85845202, -8.09933539, -2.85043906,
          0.        ,  0.        ,  0.        ,  0.        ]]),
 'y': array([[1., 1., 1., 1., 1., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 0., 0., 0., 0., 0., 0., 1., 1., 1., 1., 1., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 